In [ ]:
path_to_docs = './../data/docs_latest.tsv'

In [ ]:
import pandas as pd
import numpy as np
import re
import os

In [ ]:
!pip install pymorphy2
!pip install nltk

In [ ]:
import pymorphy2
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
morph = pymorphy2.MorphAnalyzer()

from tqdm import notebook

nltk.download("stopwords")
# некоторые стоп слова за таковые не считаем, на всякий случай
# иначе что отвечать на запрос "ЧТО ГДЕ КОГДА"?
my_stopwords = list(set(stopwords.words('russian')) - set(['как', 'когда', 'почему', 'зачем', 'чтобы', 'что']))
my_stopwords[:5]

In [ ]:
# Кеширование очень сильно ускоряет парсинг (примерно в два раза)
PYMORPHY_CACHE = {}
def lemmatizer(words):
    global PYMORPHY_CACHE
    for word in words:
        word_hash = hash(word)
        if word_hash not in PYMORPHY_CACHE:
            PYMORPHY_CACHE[word_hash] = morph.parse(word)[0].normal_form
        yield PYMORPHY_CACHE[word_hash]

In [ ]:
def process_doc(i, doc_text):
    docwords = str(doc_text[0]).lower().split()
    docwords = [word for word in docwords if word not in my_stopwords 
                   and word not in stopwords.words('english')]
    doc_proccessed = ' '.join(lemmatizer(docwords))

    path_to_save = './parsed/{0}'.format(i)
    with open(path_to_save, 'w+') as f:
        f.write(doc_proccessed)
    return doc_lemmatized

In [ ]:
!mkdir parsed

In [ ]:
from tqdm import notebook
chunksize = 100
docs_latest_path = path_to_docs
doc_ids = []
titles = []
for i, chunk in enumerate(notebook.tqdm(pd.read_csv(docs_latest_path, chunksize=chunksize, header=None, sep='\t'))):
    values = chunk.values
    doc_id, title = values[:, 0], values[:, 1]
    #doc_id, title = chunk[0], chunk[1]
    doc_ids.extend(doc_id)
    titles.extend(title)
    #if i == 2:
    #    break
    process_doc(i, test_chunk[2].values)

In [ ]:
pd.DataFrame({'DocumentId': doc_ids, 'Title': titles}).to_csv('docs_latest_titles.tsv')